# [Mediobanca 2024]()

- https://www.areastudimediobanca.com/it/product/le-medie-imprese-industriali-italiane-2012-2021

In [1]:
import os
import pandas as pd
import glob
os.chdir('D:/files/csv/Mediobanca/2024')

In [2]:
DF = pd.DataFrame()

def rename_col_by_index(dataframe, index_mapping):
    dataframe.columns = [index_mapping.get(i, col) for i, col in enumerate(dataframe.columns)]
    return dataframe

new_column_mapping = {0: 'PRN', 1: 'voce', 2:'2019', 3:'2020', 4:'2021', 5:'2022'} # Renaming columns using the function

for file_name in glob.glob('*.PRN'):
    # read the CSV file into a new DataFrame
    df = pd.read_csv(file_name, sep=',')
    if '2022' in df:
        # print("Column '2022' is present in the DataFrame", file_name)
        df = pd.read_csv(file_name, sep=',', skiprows=1)
        df = pd.concat([df.iloc[:,0:2],df.iloc[:,-4:]],axis = 1)
        df = rename_col_by_index(df, new_column_mapping) 
        df.loc[0:41,'tipo'] = 'Tab. I   - Stato patrimoniale'
        df.loc[42:87,'tipo'] = 'Tab. II  - Dati aggiuntivi'
        df.loc[88:139,'tipo'] = 'Tab. III - Conto economico'
        DF = pd.concat([DF, df], ignore_index=True)
DF = DF.dropna()

In [ ]:
PRN = pd.read_csv('decodifiche.csv', sep='|', encoding='utf-8')
MI = pd.merge( PRN[['area','settore','PRN']],DF, how = 'inner', on="PRN")
MI.shape

In [7]:
MI.sample(5)

,area,settore,PRN,voce,2019,2020,2021,2022,tipo
20714,Mezzogiorno,Legno e mobili,99001763,PASSIVO CORRENTE .........................(C),290909,296036,351854,459469,Tab. I - Stato patrimoniale
22715,Mezzogiorno,Settori del made in Italy apparten. ad altri SPL,99001751,SERVIZI,-259802,-265241,-308419,-315728,Tab. III - Conto economico
6017,Italia,Alimentare - insieme chiuso 1996-2022,88005202,SERVIZI,-1964243,-1963046,-2088126,-2420469,Tab. III - Conto economico
4659,Italia,Metallurgico,99000315,PLUSVALENZE DA CONFERIMENTO,0,0,0,0,Tab. II - Dati aggiuntivi
4444,Italia,Mezzi di trasporto e accessori,99000313,SERVIZI,-1256583,-1018981,-1321587,-1538644,Tab. III - Conto economico


In [9]:
# unpivot
melted_df = pd.melt(MI, id_vars=['area', 'settore', 'PRN','voce', 'tipo'], value_vars=['2019', '2020', '2021','2022'],
                    var_name='anno', value_name='valore')
melted_df.shape

(92928, 7)

In [11]:
melted_df.sample(5)

,area,settore,PRN,voce,tipo,anno,valore
71387,Italia,Campania,99000287,NUMERO DIPENDENTI,Tab. III - Conto economico,2022,18742
70359,Italia,Trentino-Alto Adige,99000279,APPORTI E ACQUISIZIONI,Tab. II - Dati aggiuntivi,2022,0
52294,Italia,Totale 2330 società - insieme chiuso 2017-2022,88002331,ALTRE PASSIVITA' CORRENTI,Tab. I - Stato patrimoniale,2021,12461558
12625,NordEst,Legno e mobili,99000443,CAPITALE NETTO TANGIBILE/DEBITI FINANZIARI,Tab. II - Dati aggiuntivi,2019,"145,90"
69881,Italia,Piemonte e Valle d'Aosta,99000275,F.DI AMM. RELAT. A CESPITI CONFERITI,Tab. II - Dati aggiuntivi,2022,0


In [31]:
MI.to_csv('D:/MI2024.csv', sep='|',index=False)